<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/example_kaggle/les_7/les_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505174 sha256=bd5b0aa6a205cf1453a84bf6a31237a9cdfbeff96c70344b962659ff65013fa4
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
import numpy as np
import pandas as pd

from surprise import accuracy, Dataset, Reader, SVD
from surprise.model_selection import cross_validate, train_test_split


---
# 1. Завантаження даних
---

In [5]:
import urllib.request
import os

url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
filename = "ml-latest-small.zip"

# Завантажуємо файл
urllib.request.urlretrieve(url, filename)

# Розпаковуємо
import zipfile

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(".")

print("Файл завантажено і розпаковано.")

# List extracted files
print(os.listdir("ml-latest-small"))

# Load ratings.csv
df = pd.read_csv("ml-latest-small/ratings.csv")

# Display first few rows
df.head()

Файл завантажено і розпаковано.
['links.csv', 'README.txt', 'movies.csv', 'tags.csv', 'ratings.csv']


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
# Налаштовуємо формат даних для Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

# Розділяємо дані на тренувальний та тестовий набори
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

---
# 2. Тренування моделі
---

In [7]:
# Тренуємо модель SVD
model = SVD()
model.fit(trainset)


---
# 3. Вирішення Cold Start (новий користувач)
---

Якщо в нас з’являється новий користувач, ми можемо порекомендувати йому найпопулярніші фільми або кращі за середнім рейтингом.

In [8]:
# Рахуємо середній рейтинг для кожного фільму
movie_means = df.groupby("movieId")["rating"].mean()

# Вибираємо топ-10 найкращих фільмів
top_movies = movie_means.sort_values(ascending=False).head(10)

print("Рекомендовані фільми для нового користувача:")
print(top_movies)


Рекомендовані фільми для нового користувача:
movieId
88448     5.0
100556    5.0
143031    5.0
143511    5.0
143559    5.0
6201      5.0
102217    5.0
102084    5.0
6192      5.0
145994    5.0
Name: rating, dtype: float64


---
# 4. Вирішення Cold Start (новий фільм)
---

Якщо додано новий фільм без оцінок, можна рекомендувати його схожим користувачам за жанром або контент-орієнтованими методами.

In [9]:
# Припустимо, що у нас є новий фільм movieId = 99999
new_movie_id = 99999

# Якщо немає рейтингу, рекомендуємо цей фільм схожим користувачам
if new_movie_id not in df["movieId"].values:
    print(f"Новий фільм {new_movie_id} рекомендується користувачам, які оцінили схожі фільми!")


Новий фільм 99999 рекомендується користувачам, які оцінили схожі фільми!
